In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('news_label.xlsx', engine='openpyxl')

In [3]:
df = df.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
df['content'] = df['title'] + df['content']

In [4]:
df = df.drop(columns=['근거', 'title'])

In [5]:
df['label'] = df['label'].astype(int)
df

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


### 데이터 정제하기

In [6]:
# 문자 외의 것들 제거
df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
# 혹시 모를 다중 공백 제거
df['content'] = df['content'].str.replace(' +',' ') 
df

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2336/2956849729.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2336/2956849729.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(' +',' ')


,date,content,label
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...,1
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...,1
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...,1
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...,1
...,...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...,0
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...,1
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...,0
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0


In [7]:
train = df

In [8]:
train, test = train_test_split(train)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
test.reset_index(inplace=True)

In [9]:
train = train.drop(['index'],axis=1)

In [10]:
train

,date,content,label
0,2022.05.02,창업자 188명 지상 IR 날개 K 스타트업 등용문 우뚝 오아시스마켓 경북 세일 ...,1
1,2021.12.06,부산 울산도 샛별배송 시작 지방 새벽배송 경쟁 이어진다 마켓컬리가 부산 울산지역으...,0
2,2021.12.26,아듀 2021 코로나 스타트업 고성장 두나무 직방 컬리 당근 유니콘 올해 국내 스...,1
3,2022.05.23,과포장 상장 이제 안 통한다 이커머스 IPO 줄철회 오아시스 주간 배송 테스트 돌...,0
4,2022.03.29,800리터 양문형 냉장고도 내일 배송 G마켓의 새벽 은 다를까 누가 더 멀리 가나...,1
...,...,...,...
204,2022.07.14,운영자금 고갈 투자자 압박 쏘카 오일뱅크 등 상장 강행하는 이유 교보생명 현대오일...,0
205,2022.03.13,마켓컬리 쿠팡 긴장해야겠네 네이버도 새벽배송 시장 본격 참전 올들어 잇달아 서비스...,0
206,2022.03.07,단독 이마트 퀵커머스 진출 강남대로에 MFC 1호점 연다 옛 일렉트로마트 논현점 ...,0
207,2022.02.19,20년만에 매각되는 초록마을 이마트에브리데이 컬리도 인수 군침 유통사부터 스타트업...,1


In [11]:
test = test.drop(['index'],axis=1)#.drop(['index'],axis=1) 

In [12]:
test

,date,content,label
0,2021.11.28,단군 이래 IPO 최대 라는 LG엔솔 얼마나 크길래 김민석의 IPO브리핑 시가총액...,1
1,2022.03.24,마켓컬리 드디어 출격 이커머스 3대장 IPO 시동건다 마켓컬리 오아시스마켓 실적 ...,1
2,2021.12.12,2021결산 커머스 비대면 시대 위한 M A 합종연횡 격랑 속 이커머스 LG엔솔 ...,1
3,2021.11.01,SSG 오아시스 컬리 새벽배송 업계 불붙은 IPO 레이스 상장 1호는 새벽배송 빅...,1
4,2022.07.04,하반기 IPO 대어 줄줄이 출격한다 침체된 주식시장 구세주 될까쏘카 유니콘기업 최...,1
...,...,...,...
65,2021.12.27,올해 IPO 20조 사상최대 임인년 30조 폭발 호랑이 기운 받는다 G마켓 옥션 ...,1
66,2022.05.31,오아시스마켓 맛집 그대로 세상 맛집들의 모임 이젠 빵집까지 간편식의 세계 K OT...,1
67,2022.06.08,IT클로즈업 오아시스마켓 신선식품 연합군 결성한 이유는 G 브리핑 위메이드 글로벌...,1
68,2022.04.21,빅딜 쓸어담는 NH투자증권 SK에코플랜트 주관사 낙점 대주주 사익추구 위한 쪼개기...,0


from sklearn.model_selection import train_test_split
X = df['content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train

X_test

### 토큰화 및 TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화

In [13]:
# sklearn에서 배웠던 분류 모델들을 불러와봅니다.
from sklearn.linear_model import SGDClassifier      # 1. Linear Classifier
from sklearn.linear_model import LogisticRegression # 2. Logistic Regression
from sklearn.tree import DecisionTreeClassifier     # 3. Decision Tree
from sklearn.ensemble import RandomForestClassifier # 4. Random Forest

# 평가 지표
from sklearn.metrics import accuracy_score

In [15]:
X = train['content']
y = train.label

In [20]:
max_features = 3000
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

X_vec = vectorizer.fit_transform(X)

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_vec, y)
pred = lgs.predict(X_vec)
print('Logistic Regression 학습 성능 : %.4f' % accuracy_score(y, pred))


Logistic Regression 학습 성능 : 0.8852


In [ ]:
max_features = 3000

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

X = train['content']
y = np.array(train.label)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

# evals 변수 생성
vals = [(X_val, y_val)]

# lin_reg = LinearRegression()
lgs = LogisticRegression(class_weight='balanced') # 

# lin_reg.fit(X_train, y_train)
lgs.fit(X_train, y_train)

# predicted = lin_reg.predict(X_val)
predicted = lgs.predict(X_val)

#     print(f"Linear Regression      max_features: {max_features}, Accuracy: {lin_reg.score(X_val, y_val)}" )
print(f"Logistic Regression      max_features: {max_features}, Accuracy: {lgs.score(X_val, y_val)}" )
# print(f1_score())


In [ ]:
lgs.get_params()

In [ ]:
y_val.size

In [ ]:
# Scikit-learn Wrapper XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

# evals 변수 생성
vals = [(X_val, y_val)]

# XGBClassifier 클래스의 객체 생성
# xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)
# xgb_wrapper = XGBClassifier(learning_rate= 0.1, max_depth= 3, n_estimators= 200)

# 모델 학습
# xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=400, eval_set=vals, eval_metric="logloss",  verbose=True)

# 예측
# pred = xgb_wrapper.predict(X_val)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy), '\n')


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'C': [0.1, 0.01, 0.001],
    'max_iter' : [3, 5, 7],
    'n_jobs' : [200, 400, 600]
}

grid_cv = GridSearchCV(lgs, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(X_train, y_train, eval_set=vals, eval_metric="logloss", verbose=True)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)


In [ ]:
grid_cv.get_params()

In [ ]:
for max_features in range(100, 1000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
    
    X_train = vectorizer.fit_transform(X_train)
    X_val = vectorizer.transform(X_val)
    
    lin_reg = LinearRegression()
    lgs = LogisticRegression(class_weight='balanced') # 
    
    lin_reg.fit(X_train, y_train)
    lgs.fit(X_train, y_train)
    
    predicted = lin_reg.predict(X_val)
    predicted = lgs.predict(X_val)
    
#     print(f"Linear Regression      max_features: {max_features}, Accuracy: {lin_reg.score(X_val, y_val)}" )
    print(f"Logistic Regression      max_features: {max_features}, Accuracy: {lgs.score(X_val, y_val)}" )
    print(f1_score(X_val, y_val, average='macro'))
    

In [ ]:
from sklearn.metrics import mean_squared_error

y_train = lin_reg.predict(y_test)

In [ ]:
# tf-idf 벡터화
for max_features in range(100, 10000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)

    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

    X_train = vectorizer.fit_transform(X_train)
    X_eval = vectorizer.transform(X_eval)
    
    lgs = LogisticRegression(class_weight='balanced')
    naive = BernoulliNB()
    sgd = SGDClassifier(random_state=42, n_jobs=-1)
    rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
    svc = SVC(random_state=42)
    ada = AdaBoostClassifier(random_state=42)
    
    lgs.fit(X_train, y_train)
    naive.fit(X_train, y_train)
    sgd.fit(X_train, y_train)
    rfc.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    ada.fit(X_train, y_train)
    
    predicted = lgs.predict(X_eval)
    predicted = naive.predict(X_eval)
    predicted = sgd.predict(X_eval)
    predicted = rfc.predict(X_eval)
    predicted = svc.predict(X_eval)
    predicted = ada.predict(X_eval)
    
    print(f"1. Logistic_Regression      max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )
    print(f"2. Naive_bayse             max_features: {max_features}, Accuracy: {naive.score(X_eval, y_eval)}" )
    print(f"3. SGD                     max_features: {max_features}, Accuracy: {sgd.score(X_eval, y_eval)}" )
    print(f"4. RandomForestClassifier  max_features: {max_features}, Accuracy: {rfc.score(X_eval, y_eval)}" )
    print(f"5. SVC                     max_features: {max_features}, Accuracy: {svc.score(X_eval, y_eval)}" )
    print(f"6. AdaBoost                max_features: {max_features}, Accuracy: {ada.score(X_eval, y_eval)}" )
    print('=======')

In [ ]:
from xgboost import XGBClassifier

X = train['content']
y = np.array(train.label)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=2)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

# XGBClassifier 클래스의 객체 생성
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)

# 모델 학습
xgb.fit(X_train, y_train, early_stopping_rounds=400, eval_set=vals, eval_metric="logloss",  verbose=True)

# 예측
predicted = xgb_wrapper.predict(X_val)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy))


In [ ]:
# 2600 - 3000 이 0.83으로 정확도가 가장 높게 나옴

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=2600)

X = train['content']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['content'])
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

X_train = vectorizer.fit_transform(train['content'])
X_test = vectorizer.transform(test['content'])
y = np.array(train.label)

# lgs = LogisticRegression(class_weight='balanced')
# lgs.fit(X_train, y)
# predicted = lgs.predict(X_test)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y)
predicted = rfc.predict(X_test)

In [ ]:
predicted

In [ ]:
a = predicted.tolist()
b = np.array(test.label).tolist()

In [ ]:
c = [i for i,j in zip(a,b) if i == j]
print(len(c))

In [ ]:
correct_ratio = len(c) / len(test) * 100
correct_ratio

In [ ]:
news_feature = df.drop(columns = ['content'])

In [ ]:
news_feature.to_csv('news_feature.csv')